In [41]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import os, gc
import sys

from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score

from sklearn.metrics import log_loss

from catboost import CatBoostRegressor

from datetime import datetime

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import lightgbm as lgb
import bisect
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import seaborn as sns

train = pd.read_csv('./data/train_P0.csv')

In [42]:
def mse(x_hat, x):
    # |x_hat| = (batch_size, dim)
    # |x| = (batch_size, dim)
    y = ((x - x_hat)**2).mean()
    
    return y

loss = mse

# 결측치 catboost로 채우기

채워야할 칼럼들:
1. U_WIND
2. V_WIND
3. AIR_TEMPERATURE
4. BN
5. B

채우는 방법:  

1~4 feature:    
a. 예측하려는 1개 feature를 제외한 나머지 칼럼은 drop하고, 결측치를 test data로 둠.  
b. a를 통해 예측된 feature는 다른 feature를 예측하는데 쓴다. (ex: AIR_TEMPERATURE 예측했으면, 그것을 이제 다른 feature들 예측할 때 활용)  
 -> AIR_TEMPERATURE부터 예측 진행. 왜냐면 1, 2, 3, 5는 모두 바람과 관련된 요인들이기 때문에 가장 관련성이 떨어짐.  
c. 예측은 catboost를 통해 예측

5 feature:  
채운 BN을 활용해서 공식 사용  

In [43]:
# train1['V_WIND'].fillna(train1['V_WIND'].median(), inplace=True)
# train1['AIR_TEMPERATURE'].fillna(train1['AIR_TEMPERATURE'].median(), inplace=True)
# train1['BN'].fillna(train1['BN'].median(), inplace=True)
# train1['B'].fillna(train1['B'].median(), inplace=True)

# AIR_TEMPERATURE 결측치 채우기

채워야할 칼럼들:
1. U_WIND
2. V_WIND
3. AIR_TEMPERATURE
4. BN
5. B

-> 3번을 제외한 1 2 4 5 drop하기.  

train: AIR_TEMPERATURE 가 결측치 아닌 data
test: AIR_TEMPERATURE 가 결측치인 data

In [44]:
train1 = train[:]
train1.isna().sum()

ARI_CO                     0
ARI_PO                     0
SHIP_TYPE_CATEGORY         0
DIST                       0
ID                         0
BREADTH                    0
BUILT                      0
DEADWEIGHT                 0
DEPTH                      0
DRAUGHT                    0
GT                         0
LENGTH                     0
SHIPMANAGER                0
FLAG                       0
U_WIND                153486
V_WIND                153486
AIR_TEMPERATURE       154365
BN                    153486
ATA_LT                     0
DUBAI                      0
BRENT                      0
WTI                        0
BDI_ADJ                    0
PORT_SIZE                  0
CI_HOUR                    0
DENSITY                    0
B                     153486
YEAR                       0
MONTH                      0
HOUR                       0
dtype: int64

In [45]:
train1 = train1.drop(labels=['U_WIND', 'V_WIND', 'BN', 'B'], axis=1)

In [46]:
col_list = (list(train1.columns))

In [47]:
train1.isna().sum()

ARI_CO                     0
ARI_PO                     0
SHIP_TYPE_CATEGORY         0
DIST                       0
ID                         0
BREADTH                    0
BUILT                      0
DEADWEIGHT                 0
DEPTH                      0
DRAUGHT                    0
GT                         0
LENGTH                     0
SHIPMANAGER                0
FLAG                       0
AIR_TEMPERATURE       154365
ATA_LT                     0
DUBAI                      0
BRENT                      0
WTI                        0
BDI_ADJ                    0
PORT_SIZE                  0
CI_HOUR                    0
DENSITY                    0
YEAR                       0
MONTH                      0
HOUR                       0
dtype: int64

In [48]:
test1 = train1[train1.AIR_TEMPERATURE.isna()]
train1 = train1[~train1.AIR_TEMPERATURE.isna()]

In [49]:
col_list_ = []
for c in col_list:
    if c == 'AIR_TEMPERATURE':
        continue
    col_list_.append(c)
    
train_x = train1[col_list_]
train_y = train1['AIR_TEMPERATURE']

test_x = test1[col_list_]

In [50]:
# object형 변수는 cat_features에 추가
cat_features = [f for f in train_x.columns if train_x[f].dtype == 'object']

def column_index(df, cat_features):
    cols = df.columns.values
    sidx = np.argsort(cols)
    return sidx[np.searchsorted(cols, cat_features, sorter=sidx)]

cat_features_idx = column_index(train_x, cat_features)    
print("Cat features are: %s" % [f for f in cat_features])
print(cat_features_idx)

Cat features are: ['ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'ID', 'SHIPMANAGER', 'FLAG']
[ 0  1  2  4 12 13]


In [86]:
def cat_kfold(max_depth, learning_rate, random_seed):
    cat_models={}
    
    folds=KFold(n_splits=5, shuffle=True, random_state=55)
    outcomes=[]
    sub=np.zeros((test_x.shape[0]))  
    
    for seed in random_seed:
        for n_fold, (train_index, val_index) in enumerate(folds.split(train_x, train_y)):
            print(f'===================================={n_fold + 1}============================================')
            
            X_train, X_val = train_x.iloc[train_index], train_x.iloc[val_index]
            y_train, y_val = train_y.iloc[train_index], train_y.iloc[val_index]

            # early_stopping 50에서 가장 좋은 점수를 내는 learning_rate를 활용
            cat = CatBoostRegressor(n_estimators=5000, max_depth=max_depth, random_seed=seed, custom_metric=['RMSE', 'MAE', 'R2'], learning_rate=learning_rate, loss_function='RMSE', bootstrap_type ='Bernoulli')
            cat.fit(X_train, y_train,
                  eval_set=[(X_train, y_train), (X_val, y_val)],
                  early_stopping_rounds=50, cat_features=cat_features,
                  verbose=100)

            cat_models[n_fold] = cat

            # val 데이터 예측
            predictions = cat.predict(X_val)
            # test 데이터 예측
            test_predictions = cat.predict(test_x)

            # val 데이터 예측 logloss 값 저장
            logloss=mse(y_val, predictions)
            outcomes.append(logloss)
            print(f"FOLD {n_fold + 1} : mse:{logloss}")
            
            print(cat.get_best_score())

            # test 데이터 예측 결과 종합
            # 최종 적으로는 kolds 횟수 만큼 나눠서 평균 값을 활용
            print(sub)
            print(test_predictions)
            sub += test_predictions
            print(sub)

            print(f'================================================================================\n\n')

    # 저장된 val 데이터 예측 logloss 값의 평균 값으로 성능을 비교
    mean_outcome=np.mean(outcomes)
    print("Mean:{}".format(mean_outcome))
    
    return sub / (folds.n_splits * len(random_seed))

In [69]:
my_submission = cat_kfold(8, 0.04, [1042])

====================================1============================================
0:	learn: 9.4217100	test: 9.4211241	test1: 9.4494649	best: 9.4494649 (0)	total: 457ms	remaining: 38m 4s
100:	learn: 3.6724449	test: 3.6378828	test1: 3.6548177	best: 3.6548177 (100)	total: 32.7s	remaining: 26m 27s
200:	learn: 3.3849356	test: 3.3477766	test1: 3.3760235	best: 3.3760235 (200)	total: 1m 3s	remaining: 25m 24s
300:	learn: 3.2485111	test: 3.2134240	test1: 3.2483510	best: 3.2483510 (300)	total: 1m 34s	remaining: 24m 28s
400:	learn: 3.1418444	test: 3.1093580	test1: 3.1511499	best: 3.1511499 (400)	total: 2m 4s	remaining: 23m 46s
500:	learn: 3.0631211	test: 3.0337866	test1: 3.0819021	best: 3.0819021 (500)	total: 2m 34s	remaining: 23m 4s
600:	learn: 2.9935700	test: 2.9662060	test1: 3.0208616	best: 3.0208616 (600)	total: 3m 4s	remaining: 22m 28s
700:	learn: 2.9377142	test: 2.9123902	test1: 2.9727661	best: 2.9727661 (700)	total: 3m 34s	remaining: 21m 55s
800:	learn: 2.8918751	test: 2.8690373	test1: 2.93

In [70]:
my_submission

array([26.30144597, 10.22391838, 30.05551694, ...,  3.22531455,
       23.65034502,  7.76846577])

In [71]:
test1.AIR_TEMPERATURE = my_submission

In [74]:
test1.index

Int64Index([     5,      8,      9,     10,     11,     12,     13,     15,
                20,     22,
            ...
            367418, 367420, 367422, 367423, 367425, 367427, 367428, 367432,
            367435, 367436],
           dtype='int64', length=154365)

In [81]:
train.loc[test1.index, "AIR_TEMPERATURE"] = my_submission

In [82]:
train.iloc[test1.index].AIR_TEMPERATURE

5         26.301446
8         10.223918
9         30.055517
10        25.367937
11        28.026101
            ...    
367427     8.010137
367428     9.576394
367432     3.225315
367435    23.650345
367436     7.768466
Name: AIR_TEMPERATURE, Length: 154365, dtype: float64

In [84]:
train.isna().sum()

ARI_CO                     0
ARI_PO                     0
SHIP_TYPE_CATEGORY         0
DIST                       0
ID                         0
BREADTH                    0
BUILT                      0
DEADWEIGHT                 0
DEPTH                      0
DRAUGHT                    0
GT                         0
LENGTH                     0
SHIPMANAGER                0
FLAG                       0
U_WIND                153486
V_WIND                153486
AIR_TEMPERATURE            0
BN                    153486
ATA_LT                     0
DUBAI                      0
BRENT                      0
WTI                        0
BDI_ADJ                    0
PORT_SIZE                  0
CI_HOUR                    0
DENSITY                    0
B                     153486
YEAR                       0
MONTH                      0
HOUR                       0
dtype: int64

In [88]:
train2 = train[:]
train2.isna().sum()

train2 = train2.drop(labels=['U_WIND', 'V_WIND', 'B'], axis=1)

col_list = (list(train2.columns))

test2 = train2[train2.BN.isna()]
train2 = train2[~train2.BN.isna()]

col_list_ = []
for c in col_list:
    if c == 'BN':
        continue
    col_list_.append(c)
    
train_x = train2[col_list_]
train_y = train2['BN']

test_x = test2[col_list_]

# object형 변수는 cat_features에 추가
cat_features = [f for f in train_x.columns if train_x[f].dtype == 'object']

def column_index(df, cat_features):
    cols = df.columns.values
    sidx = np.argsort(cols)
    return sidx[np.searchsorted(cols, cat_features, sorter=sidx)]

cat_features_idx = column_index(train_x, cat_features)    
print("Cat features are: %s" % [f for f in cat_features])
print(cat_features_idx)

cat_models2={}

my_submission2 = cat_kfold(8, 0.04, [1042])

Cat features are: ['ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'ID', 'SHIPMANAGER', 'FLAG']
[ 0  1  2  4 12 13]
====================================1============================================
0:	learn: 1.3647291	test: 1.3642586	test1: 1.3619782	best: 1.3619782 (0)	total: 705ms	remaining: 58m 41s
100:	learn: 1.0005253	test: 0.9948539	test1: 0.9935865	best: 0.9935865 (100)	total: 37s	remaining: 29m 57s
200:	learn: 0.9749278	test: 0.9690541	test1: 0.9702870	best: 0.9702870 (200)	total: 1m 16s	remaining: 30m 36s
300:	learn: 0.9584435	test: 0.9527407	test1: 0.9562774	best: 0.9562774 (300)	total: 1m 50s	remaining: 28m 39s
400:	learn: 0.9424819	test: 0.9368278	test1: 0.9431013	best: 0.9431013 (400)	total: 2m 23s	remaining: 27m 26s
500:	learn: 0.9300015	test: 0.9243881	test1: 0.9331789	best: 0.9331789 (500)	total: 2m 57s	remaining: 26m 30s
600:	learn: 0.9202278	test: 0.9146233	test1: 0.9263882	best: 0.9263882 (600)	total: 3m 32s	remaining: 25m 52s
700:	learn: 0.9111934	test: 0.9058889	test1: 0

In [92]:
test2.BN

5        NaN
8        NaN
9        NaN
10       NaN
11       NaN
          ..
367427   NaN
367428   NaN
367432   NaN
367435   NaN
367436   NaN
Name: BN, Length: 153486, dtype: float64

In [93]:
print(my_submission2)

test2.BN = my_submission2

test2.BN

[ 0.350414   -0.39763456 -0.06456282 ...  0.18059043  1.57857309
  1.7551912 ]


5         0.350414
8        -0.397635
9        -0.064563
10        2.821210
11        3.103631
            ...   
367427    2.023370
367428    1.469253
367432    0.180590
367435    1.578573
367436    1.755191
Name: BN, Length: 153486, dtype: float64

In [94]:
train.loc[test2.index, "BN"] = my_submission2

In [95]:
train.isna().sum()

ARI_CO                     0
ARI_PO                     0
SHIP_TYPE_CATEGORY         0
DIST                       0
ID                         0
BREADTH                    0
BUILT                      0
DEADWEIGHT                 0
DEPTH                      0
DRAUGHT                    0
GT                         0
LENGTH                     0
SHIPMANAGER                0
FLAG                       0
U_WIND                153486
V_WIND                153486
AIR_TEMPERATURE            0
BN                         0
ATA_LT                     0
DUBAI                      0
BRENT                      0
WTI                        0
BDI_ADJ                    0
PORT_SIZE                  0
CI_HOUR                    0
DENSITY                    0
B                     153486
YEAR                       0
MONTH                      0
HOUR                       0
dtype: int64

In [96]:
def calculate_B_from_V(V):
    B = (V / 0.836) ** (2/3)
    return B

In [97]:
train['B'] = train.BN.apply(lambda x:calculate_B_from_V(x))

In [98]:
train.isna().sum()

ARI_CO                     0
ARI_PO                     0
SHIP_TYPE_CATEGORY         0
DIST                       0
ID                         0
BREADTH                    0
BUILT                      0
DEADWEIGHT                 0
DEPTH                      0
DRAUGHT                    0
GT                         0
LENGTH                     0
SHIPMANAGER                0
FLAG                       0
U_WIND                153486
V_WIND                153486
AIR_TEMPERATURE            0
BN                         0
ATA_LT                     0
DUBAI                      0
BRENT                      0
WTI                        0
BDI_ADJ                    0
PORT_SIZE                  0
CI_HOUR                    0
DENSITY                    0
B                          0
YEAR                       0
MONTH                      0
HOUR                       0
dtype: int64

In [100]:
train3 = train[:]

train3 = train3.drop(labels=['V_WIND'], axis=1)

col_list = (list(train3.columns))

test3 = train3[train3.U_WIND.isna()]
train3 = train3[~train3.U_WIND.isna()]

col_list_ = []
for c in col_list:
    if c == 'U_WIND':
        continue
    col_list_.append(c)
    
train_x = train3[col_list_]
train_y = train3['U_WIND']

test_x = test3[col_list_]

# object형 변수는 cat_features에 추가
cat_features = [f for f in train_x.columns if train_x[f].dtype == 'object']

def column_index(df, cat_features):
    cols = df.columns.values
    sidx = np.argsort(cols)
    return sidx[np.searchsorted(cols, cat_features, sorter=sidx)]

cat_features_idx = column_index(train_x, cat_features)    
print("Cat features are: %s" % [f for f in cat_features])
print(cat_features_idx)

my_submission3 = cat_kfold(8, 0.04, [1042])

Cat features are: ['ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'ID', 'SHIPMANAGER', 'FLAG']
[ 0  1  2  4 12 13]
====================================1============================================


c:\Users\kjsjh\anaconda3\lib\site-packages\catboost\core.py:1411: ComplexWarning: Casting complex values to real discards the imaginary part
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


0:	learn: 3.3376185	test: 3.3368386	test1: 3.3207611	best: 3.3207611 (0)	total: 426ms	remaining: 35m 28s
100:	learn: 2.6837805	test: 2.6637599	test1: 2.6705953	best: 2.6705953 (100)	total: 38.6s	remaining: 31m 14s
200:	learn: 2.5738417	test: 2.5517845	test1: 2.5724923	best: 2.5724923 (200)	total: 1m 14s	remaining: 29m 49s
300:	learn: 2.5017875	test: 2.4788830	test1: 2.5117688	best: 2.5117688 (300)	total: 1m 50s	remaining: 28m 48s
400:	learn: 2.4387120	test: 2.4157652	test1: 2.4602438	best: 2.4602438 (400)	total: 2m 26s	remaining: 28m 3s
500:	learn: 2.3873529	test: 2.3648739	test1: 2.4192887	best: 2.4192887 (500)	total: 3m 1s	remaining: 27m 12s
600:	learn: 2.3489029	test: 2.3267417	test1: 2.3903160	best: 2.3903160 (600)	total: 3m 37s	remaining: 26m 30s
700:	learn: 2.3130551	test: 2.2920758	test1: 2.3652741	best: 2.3652741 (700)	total: 4m 12s	remaining: 25m 48s
800:	learn: 2.2817963	test: 2.2618745	test1: 2.3444950	best: 2.3444950 (800)	total: 4m 47s	remaining: 25m 6s
900:	learn: 2.25242

In [103]:
print(my_submission2)

test3.U_WIND = my_submission3

test3.U_WIND

[ 0.350414   -0.39763456 -0.06456282 ...  0.18059043  1.57857309
  1.7551912 ]


5        -0.206099
8         0.536769
9        -0.075533
10       -0.366430
11        1.323759
            ...   
367427    0.051511
367428    0.418191
367432   -0.048267
367435    0.120379
367436    0.378498
Name: U_WIND, Length: 153486, dtype: float64

In [105]:
train.loc[test3.index, "U_WIND"] = my_submission3

In [106]:
train.isna().sum()

ARI_CO                     0
ARI_PO                     0
SHIP_TYPE_CATEGORY         0
DIST                       0
ID                         0
BREADTH                    0
BUILT                      0
DEADWEIGHT                 0
DEPTH                      0
DRAUGHT                    0
GT                         0
LENGTH                     0
SHIPMANAGER                0
FLAG                       0
U_WIND                     0
V_WIND                153486
AIR_TEMPERATURE            0
BN                         0
ATA_LT                     0
DUBAI                      0
BRENT                      0
WTI                        0
BDI_ADJ                    0
PORT_SIZE                  0
CI_HOUR                    0
DENSITY                    0
B                          0
YEAR                       0
MONTH                      0
HOUR                       0
dtype: int64

In [107]:
train4 = train[:]

col_list = (list(train4.columns))

test4 = train4[train4.V_WIND.isna()]
train4 = train4[~train4.V_WIND.isna()]

col_list_ = []
for c in col_list:
    if c == 'V_WIND':
        continue
    col_list_.append(c)
    
train_x = train4[col_list_]
train_y = train4['V_WIND']

test_x = test4[col_list_]

# object형 변수는 cat_features에 추가
cat_features = [f for f in train_x.columns if train_x[f].dtype == 'object']

def column_index(df, cat_features):
    cols = df.columns.values
    sidx = np.argsort(cols)
    return sidx[np.searchsorted(cols, cat_features, sorter=sidx)]

cat_features_idx = column_index(train_x, cat_features)    
print("Cat features are: %s" % [f for f in cat_features])
print(cat_features_idx)

my_submission4 = cat_kfold(8, 0.04, [1042])

Cat features are: ['ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'ID', 'SHIPMANAGER', 'FLAG']
[ 0  1  2  4 12 13]
====================================1============================================


c:\Users\kjsjh\anaconda3\lib\site-packages\catboost\core.py:1411: ComplexWarning: Casting complex values to real discards the imaginary part
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


0:	learn: 3.6878239	test: 3.6866095	test1: 3.6789697	best: 3.6789697 (0)	total: 444ms	remaining: 37m
100:	learn: 2.5615175	test: 2.5300371	test1: 2.5407580	best: 2.5407580 (100)	total: 42.1s	remaining: 34m
200:	learn: 2.4126120	test: 2.3794626	test1: 2.4022568	best: 2.4022568 (200)	total: 1m 17s	remaining: 30m 51s
300:	learn: 2.3328033	test: 2.2999016	test1: 2.3325025	best: 2.3325025 (300)	total: 1m 53s	remaining: 29m 34s
400:	learn: 2.2680813	test: 2.2353877	test1: 2.2786730	best: 2.2786730 (400)	total: 2m 29s	remaining: 28m 33s
500:	learn: 2.2110573	test: 2.1789442	test1: 2.2311345	best: 2.2311345 (500)	total: 3m 5s	remaining: 27m 43s
600:	learn: 2.1676113	test: 2.1367799	test1: 2.1996767	best: 2.1996767 (600)	total: 3m 41s	remaining: 26m 57s
700:	learn: 2.1316996	test: 2.1026877	test1: 2.1755449	best: 2.1755449 (700)	total: 4m 16s	remaining: 26m 13s
800:	learn: 2.1000500	test: 2.0723230	test1: 2.1536032	best: 2.1536032 (800)	total: 4m 52s	remaining: 25m 31s
900:	learn: 2.0720772	tes

In [109]:
print(my_submission4)

test4.V_WIND = my_submission4

test4.V_WIND

[-0.28831174 -0.08855622 -0.26675287 ...  0.19090097 -0.07096185
  0.19455084]


5        -0.288312
8        -0.088556
9        -0.266753
10        2.055102
11       -0.736205
            ...   
367427   -1.321900
367428    0.653867
367432    0.190901
367435   -0.070962
367436    0.194551
Name: V_WIND, Length: 153486, dtype: float64

In [110]:
train.isna().sum()

ARI_CO                     0
ARI_PO                     0
SHIP_TYPE_CATEGORY         0
DIST                       0
ID                         0
BREADTH                    0
BUILT                      0
DEADWEIGHT                 0
DEPTH                      0
DRAUGHT                    0
GT                         0
LENGTH                     0
SHIPMANAGER                0
FLAG                       0
U_WIND                     0
V_WIND                153486
AIR_TEMPERATURE            0
BN                         0
ATA_LT                     0
DUBAI                      0
BRENT                      0
WTI                        0
BDI_ADJ                    0
PORT_SIZE                  0
CI_HOUR                    0
DENSITY                    0
B                          0
YEAR                       0
MONTH                      0
HOUR                       0
dtype: int64

In [111]:
train.loc[test4.index, "V_WIND"] = my_submission4

In [112]:
train.isna().sum()

ARI_CO                0
ARI_PO                0
SHIP_TYPE_CATEGORY    0
DIST                  0
ID                    0
BREADTH               0
BUILT                 0
DEADWEIGHT            0
DEPTH                 0
DRAUGHT               0
GT                    0
LENGTH                0
SHIPMANAGER           0
FLAG                  0
U_WIND                0
V_WIND                0
AIR_TEMPERATURE       0
BN                    0
ATA_LT                0
DUBAI                 0
BRENT                 0
WTI                   0
BDI_ADJ               0
PORT_SIZE             0
CI_HOUR               0
DENSITY               0
B                     0
YEAR                  0
MONTH                 0
HOUR                  0
dtype: int64

In [113]:
train.to_csv("./data/train_P1.csv", index=False)